In [ ]:
! pip install --quiet ragstack-ai

In [ ]:
UNSTRUCTURED_API_KEY='XXXXXXXXX'
UNSTRUCTURED_API_URL='https://api.unstructured.io/general/v0/general'
ASTRA_DB_API_ENDPOINT='https://XXXXXXXXXXXX-us-east-2.apps.astra.datastax.com'
ASTRA_DB_APPLICATION_TOKEN='AstraCS:XXXXXXXX'
OPENAI_API_KEY='XXXXXXXX'

# Simple Loading of document

In [ ]:
from langchain_community.document_loaders import UnstructuredAPIFileLoader

loader = UnstructuredAPIFileLoader(
    file_path="the-power-rhonda-byrne.pdf",
    api_key='XXXX',
    url='https://api.unstructured.io/general/v0/general',
)
simple_docs = loader.load()
len(simple_docs)

1

In [ ]:
print(simple_docs[0].page_content[0:400])

P4 [ ¢ (34 o T P t ( the > OW ER 3 g £ e @ et L i 22 € < : A

o

&

o i &~

&

#

%

In this book I want to show you the way to an amazing life. There is so much for you to know about life, and it is all good. In fact, it is beyond good. It is phenomenal!

Life is so much easier than you think it is, and as you come to understand the way life works, and The Power you have inside you, you will expe


# Loading PDF with the advance parsing


*   By changing the processing strategy and **response mode**, we can get more detailed document structure. Unstructured can break the document into elements of different types, which can be helpful for improving your RAG system.
*   You can use the link that explains the method well.
https://docs.unstructured.io/examplecode/codesamples/apioss/table-extraction-from-pdf
*  A list of all the different element. Returned metadata can also be helpful. For example, the page_number of the pdf input, and a parent_id property which helps define nesting of text sections.





In [ ]:
from langchain_community.document_loaders import unstructured

elements = unstructured.get_elements_from_api(
    file_path="the-power-rhonda-byrne.pdf",
    api_key='XXXX',
    api_url='https://api.unstructured.io/general/v0/general',
    strategy="hi_res",  # default "auto"
    pdf_infer_table_structure=True,
)

len(elements)

2999

## We can see some of the elements.

In [ ]:
print(elements[0])

P4 [ ¢ (34 o T P t ( the > OW ER 3 g £ e @ et L i 22 € < : A


In [ ]:
print(elements[7])

In this book I want to show you the way to an amazing life. There is so much for you to know about life, and it is all good. In fact, it is beyond good. It is phenomenal!


# Structuring the elements and we use element type and parent_id to show a clearer representation of the document structure.

Note: Here we clearly see that Unstructured is parsing both table and document structure.

Here we clearly see that Unstructured is parsing both table and document structure.

In [ ]:
from IPython.display import HTML, display

parents = {}

for el in elements:
    parents[el.id] = el.text

for el in elements:
    if el.category == "Table":
        display(HTML(el.metadata.text_as_html))
    elif el.metadata.parent_id:
        print(f"parent: '{parents[el.metadata.parent_id]}' content: {el.text}")
    else:
        print(el)

P4 [ ¢ (34 o T P t ( the > OW ER 3 g £ e @ et L i 22 € < : A
o
parent: 'o' content: &
o i &~
parent: 'o i &~' content: &
parent: 'o i &~' content: #
parent: 'o i &~' content: %
parent: 'o i &~' content: In this book I want to show you the way to an amazing life. There is so much for you to know about life, and it is all good. In fact, it is beyond good. It is phenomenal!
parent: 'o i &~' content: Life is so much easier than you think it is, and as you come to understand the way life works, and The Power you have inside you, you will experience the magic of life in its fullness - and then you will have an amazing life!
parent: 'o i &~' content: 3 '1
parent: 'o i &~' content: Now let the magic of your life begin.
parent: 'o i &~' content: from The Power
For more information about The Power; visit www.thesecret.tv.
BODY, MIND, AND SPIRIT
g
8] is to - Without have been
523.95 u.s. 526.95 Can.
, is the handbook to the greatest
powe i the universe - The Power to have everything you want.
Wit

# Now we will continue with the RAG process, by creating embeddings for the pdf.

In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

astra_db_store = AstraDBVectorStore(
    collection_name="secrets",
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
)

/usr/local/lib/python3.10/dist-packages/langchain_astradb/utils/astradb.py:248: UserWarning: Astra DB collection 'secrets' is detected as having indexing turned on for all fields (either created manually or by older versions of this plugin). This implies stricter limitations on the amount of text each string in a document can store. Consider reindexing anew on a fresh collection to be able to store longer texts.
  if not self._validate_indexing_policy(


In [ ]:
elements[20].text

'At (]'

In [ ]:
elements[21].text

'The life of your dreams has always been closer to you than you realized, because The Power - to have everything good in life - is inside you.'

# We will create LangChain Documents by splitting the text after Table elements and before Title elements. Additionally, we use the html output format for table data.

In [ ]:
from langchain_core.documents import Document

documents = []
current_doc = None

for el in elements:
    if el.category in ["Header", "Footer"]:
        continue  # skip these
    if el.category == "Title":
        documents.append(current_doc)
        current_doc = None
    if not current_doc:
        current_doc = Document(page_content="", metadata=el.metadata.to_dict())
    current_doc.page_content += (
        el.metadata.text_as_html if el.category == "Table" else el.text
    )
    if el.category == "Table":
        documents.append(current_doc)
        current_doc = None

astra_db_store.add_documents(documents)

['ca58e6d0f650406db910c1b68b4186a2',
 'a340655f93894c1385cab4cf7bf9a6e0',
 '9355a3e6f74b4862a88332a8a5b2ecc6',
 'da4287a7ed4a4ff2a82ef51a228eb8d8',
 '6f4f2231ae3a4e1390a09e2836c22777',
 '498d404ae4d34c7a91bceb56f3b92d0c',
 'b64a22cc742944dc9633f95a16abd1ab',
 'a208acf468ac4dd995dbd451446af8ac',
 '6be3dff5b2fb4700b0fff0aacd3a04d1',
 '660a24cae5fb447e8538f54b22979ccd',
 'ba59b8c9e9f848c091da2b59584884ae',
 '81fa2c25f46a4d3aae3792bbda19d7f0',
 '320c1b6c0ece411183872f4415068792',
 'e4524e56b077455c87aab4ec070635a6',
 'ad99d17149e0430391f23c1f5238a0dd',
 '7fc5839a17974b52854fe964c9e0affd',
 'fca5356b39b54fcbba2a96216cfa02f1',
 'f1c6115d22424a569dc8dfd1b6cd4864',
 '45b4ea4aff864121b6ce94545a393e16',
 '6c98ed3ab4324c99b99225c051adafda',
 '8ce7d594ad2e43a2adea1570894e866e',
 '48acb1f2134248968eb085f7c5eefdd0',
 'ed4ad69a5e1d46ad8b4dc72b36271cd9',
 'abc97f04f5aa463d9095ae15c0ca5444',
 'e79f133354b041c48cc6adaf20787634',
 '74d200ea125249ef9d449a5da41dacaf',
 '12e86e967f704e96b64686df588b9981',
 

In [ ]:
len(documents)

506

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = """
Answer the question based only on the supplied context. If you don't know the answer, say "I don't know".
Context: {context}
Question: {question}
Your answer:
"""  # noqa: E501

llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=False, temperature=0,openai_api_key=OPENAI_API_KEY)

chain = (
    {"context": astra_db_store.as_retriever(), "question": RunnablePassthrough()}
    | PromptTemplate.from_template(prompt)
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("How to get success with upset mood?")

"To get success with an upset mood, it is important to change the negative reaction right away. Simply being aware that you've reacted negatively takes the power out of the negative feelings immediately and can even stop them. If the negative feelings have a grip on you, it is best to walk away and spend a couple of minutes looking for the things you love, one after the other until you feel much better. You can use anything you love to make you feel better, like listening to your favorite music, imagining the things you love, or doing something you love. You can also think about the things you love in the person who upset you. This might be challenging, but if you can do it, it is the fastest way to feel better and become the master of your feelings."

In [ ]:
chain.invoke("Need solution for mental peace?")

'Focus on love and train your mind to think positively by asking it questions that redirect its focus. Keep asking your mind questions regularly to stay in control and prevent negative thoughts from taking over.'

In [ ]:
chain.invoke("How can I get more knowledge?")

'One way to get more knowledge is by asking your mind questions regularly and staying in control of it. By asking questions such as "What can I see that I love?" or "What can I hear that I love?", you can keep your mind focused and actively seeking answers, thus increasing your knowledge.'

In [ ]:
chain.invoke("How One could only damage oneself?")

"One could only damage oneself by negatively affecting another person, which will result in the negativity returning in full to affect one's own circumstances such as money, career, health, or relationships."

In [ ]:
chain.invoke("What to do when mind is not cool?")

'When the mind is not cool, one should ask themselves questions to redirect their focus and regain control.'

In [ ]:
chain.invoke("What is mental peace")

'Based on the context provided, mental peace refers to a state of calmness and tranquility in the mind.'

In [ ]:
chain.invoke("Need help to focus on my work?")

"I don't know."

In [ ]:
chain.invoke("How to control good feeling?")

'Amplify your good feelings by thinking about all the things you love and counting them nonstop until you feel amazing.'

In [ ]:
chain.invoke("How to control vocabulary?")

'Throw out words like terrible, horrible, disgusting, and awful, and instead use words like fantastic, amazing, fabulous, brilliant, and wonderful.'

In [ ]:
chain.invoke("what should I do when the reverse happening when something went wrong?")

'If something negative happens and your feelings drop, find an excuse to politely walk away and restore yourself with good feelings.'